init jeu de donnees et affichage des informations, on notte une difference entre les colonnes Total_Proteins de chaque table

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# A compléter
df_h1 = pd.read_json('./liver.json')
df_h2 = pd.read_csv('./liver.csv')
print(df_h1.info())
print(df_h2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         291 non-null    object 
 1   Gender                      291 non-null    object 
 2   Total_Bilirubin             290 non-null    float64
 3   Direct_Bilirubin            287 non-null    float64
 4   Alkaline_Phosphotase        290 non-null    float64
 5   Alamine_Aminotransferase    236 non-null    float64
 6   Aspartate_Aminotransferase  291 non-null    int64  
 7   Total_Proteins              289 non-null    float64
 8   Albumin                     290 non-null    float64
 9   Albumin_and_Globulin_Ratio  227 non-null    float64
 10  Outcome                     291 non-null    int64  
dtypes: float64(7), int64(2), object(2)
memory usage: 25.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (

combination donnees

avant de combiner les tables on fait l'harmonisation de la colonne Total_Proteins

In [16]:
def check_protein_format(protein_str):
    return isinstance(protein_str, str) and protein_str.strip().endswith('mg/dL')

def extract_protein_value(protein_str):
    if pd.isna(protein_str):
        return pd.NA
    if check_protein_format(protein_str):
        try:
            return float(protein_str.strip().replace('mg/dL','').strip())
        except:
            return pd.NA
    try:
        return float(protein_str/1000)
    except:
        return pd.NA

# Aplicar la función al dataframe (df_h2 ya debería existir)
df_h2['Total_Proteins'] = df_h2['Total_Proteins'].apply(extract_protein_value)
df_h2['Total_Proteins'].head()

0    7.8
1    7.1
2    7.7
3    8.3
4    8.0
Name: Total_Proteins, dtype: object

on fait l'harmonisation des unites de  Total_Bilirubin et Direct_Bilirubin

In [19]:
def harmonize_skin_thickness(df):
    df = df.copy()
    if 'Total_Bilirubin' in df.columns and 'Direct_Bilirubin' in df.columns:
        df['Total_Bilirubin'] = df['Total_Bilirubin'] / 10
        df['Direct_Bilirubin'] = df['Direct_Bilirubin'] / 10
    return df

df_h2 = harmonize_skin_thickness(df_h2)
df_h2[['Total_Bilirubin','Direct_Bilirubin']].head()

,Total_Bilirubin,Direct_Bilirubin
0,0.7,0.2
1,0.6,0.2
2,1.1,0.5
3,2.4,1.0
4,0.6,0.2


Albumin_and_Globulin_Ratio = Albumin/(Total_protein -Albumin) alors on verifie la coherence 

In [ ]:
def check_agr_consistency(df):
    df = df.copy()
    if 'Albumin' in df.columns and 'Total_protein' in df.columns and 'Albumin_and_Globulin_Ratio' in df.columns:
        agr_calc = df['Albumin'] / (df['Total_protein']-df['Albumin'])
        diff = (df['Albumin_and_Globulin_Ratio'] - agr_calc).abs()
        return diff == 0
    else:
        import pandas as pd
        return pd.Series([False]*len(df), index=df.index)


consistency = check_agr_consistency(df_combined)
consistency.value_counts()

apres revison on decide d'effacer les colonnes Albumin et Total_protein 

In [ ]:
df_combined = df_combined.drop(columns=['Albumin', 'Total_protein'], errors='ignore')
df_combined.head()